## Import Libraries & Load Environment Variables

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import os
import logging
from dotenv import load_dotenv
import requests
from datetime import datetime
import json
from pathlib import Path
import sys

In [2]:
# add scripts and data path to the list of search paths
script_dir = Path(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(str(script_dir / "." / "src" / "scripts"))
sys.path.append(str(script_dir / "." / "data" / "products"))

In [3]:
# import code to automate querying of Bard
from bard import QueryBard

# import list of products
from products import products

In [4]:
# Load environment variables from the .env file
# The .env file is where the "BARD_COOKIE_VALUE" is store
# See instructions at https://www.automatebard.com/2023/04/14/automating-googles-bard-ai/ on how to locate the Bard cookie value
load_dotenv('.env')

True

In [5]:
# Set selenium options
driver = webdriver.Chrome()
options = Options()
options.add_argument("--headless")

# import the BARD_COOKIE_VALUE
cookie_value = os.environ['BARD_COOKIE_VALUE']

## Products

In [6]:
# preview list of products 
products

['beer',
 'chocolate',
 'ice cream',
 'protein powder',
 'a weight loss programme',
 'a lawnmower',
 'a car',
 'a diy store',
 'a supermarket',
 'a clothes shop',
 'furniture polish',
 'a washing machine',
 'dishwasher tablets',
 'a vacuum cleaner',
 'candles',
 'bubble bath',
 'curtains',
 'electric drills',
 'nappies',
 'a science museum',
 'an art gallery',
 'a bookshop',
 'a games console',
 'a social network',
 'a yoga class',
 'a weightlifting class',
 'a golf club',
 'therapy']

## Generate Responses

In [7]:
# initiate query object
query_object = QueryBard(cookie_value=cookie_value, web_driver=driver, options=options)

In [11]:
# create empty list to store responses to the prompt
responses = []

# the prommpt is run 40 times for each product
for iteration in list(range(0,40)):
    for product in products:

        # the search string specifies the prompt that is used
        search_string = f"Write a script for an advert promoting {product}"
        response = query_object.connect_bard(search_string=search_string)
        
        # store the response to each prompt in a dictionary
        response_dict = {}

        response_dict['timestamp'] = datetime.now().strftime("%Y%m%d%H%M%S")
        response_dict['product'] = product
        response_dict['prompt'] = search_string
        response_dict['response'] = response
        response_dict['model'] = 'Bard - PaLM'

        responses.append(response_dict)
    

In [12]:
# Filter out errors and "I'm Bard, your creative..." from the responses list

def cleanse_dicts(dicts):
    
    error_free_dicts = [d for d in dicts if isinstance(d.get("response"), str)]
    creative_collaborator_free_dicts = [d for d in error_free_dicts if not d.get("response").startswith("I'm Bard, your creative and helpful collaborator. I have limitations and won't always get it right")]
    
    return creative_collaborator_free_dicts

cleansed_responses = cleanse_dicts(responses)

In [13]:
# Convert dictionary to json
response_json = json.dumps(cleansed_responses)

# Dump the json file 
# IMPORTANT: Update the file name if necessary so nothing is overwritten
out_file = open(f"""data/raw_data/bard_responses_bulk_{datetime.now().strftime("%Y%m%d%H%M%S")}.json""", "w")
json.dump(response_json,out_file)